In [38]:
import requests
import lxml.html as lh
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

# URL der Universitäten aus dem Top Ranking
url = "https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2019_limit0_7216a250f6ae72c71cd09563798a9f18.json"

page = requests.get(url) # GET auf URL

jsonData = page.json() # Umwandlung in JSON

gerUniversities = []

for uni in jsonData['data']:
    
    # Filtern der deutschen Universitäten und speichern in Array
    if(str(uni['location']) == 'Germany'):
        gerUniversities.append(uni)
    
#[uni['name'].replace(" ", "_") for uni in gerUniversities] # Anzeige

In [40]:
# SPARQL Anfrage auf DBpedia

sparql = SPARQLWrapper("http://dbpedia.org/sparql") # SPARQL Interpreter 

# Für jede Uni in Array
for uni in gerUniversities:
    
    uniname = uni["name"].replace(" ", "_") # Einheitliche Struktur erstellen
    #print(uniname)
    
    # Query für DBpediaseite der Universität erstellen
    sparql.setQuery("""
        SELECT (str(?nameObject) as ?name), (str(?cityObject) as ?city), (str(?countryObject) as ?country),
        (str(?numOfStudentsObject) as ?numOfStudents), ?links, ?geoPoint
        WHERE {
            <http://en.wikipedia.org/wiki/""" + uniname + """>  foaf:primaryTopic ?site.
            ?site rdfs:label ?nameObject.
            FILTER (lang(?nameObject) = "" || lang(?nameObject) = "de")
            ?site dbo:city ?cityObject.
            ?site dbo:country ?countryObject.
            ?site dbo:numberOfStudents ?numOfStudentsObject.
            ?site foaf:homepage ?homepage.
            ?site georss:point ?geoPoint
        }
    """)
    
    # Rückgabetyp setzen
    sparql.setReturnFormat("json")
    #sparql.setReturnFormat("xml")
    
    result = sparql.query().convert() # Anfrage stellen
    
    #print(result) # Anzeige

<Element html at 0x271b7f73e08>
[<Element header at 0x271b822fcc8>]
